<a href="https://colab.research.google.com/github/M000N5H1N3/M000N5H1N3/blob/main/SimpleSmsSpamFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#connect to google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data_file = '/content/drive/My Drive/Untitled document.txt'

In [15]:
import pandas as pd

data = pd.read_csv(data_file, sep=',')
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [16]:
#load stopwords and punctuation

import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')

stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

print(stopwords[:5])
print(punctuation)

['i', 'me', 'my', 'myself', 'we']
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
#pre-process sms content

def pre_process(v2):
  lowercase = "".join([char.lower() for char in v2 if char not in punctuation])
  tokenize = nltk.tokenize.word_tokenize(lowercase)
  remove_stopwords = [word for word in tokenize if word not in stopwords]
  return remove_stopwords

data['processed'] = data['v2'].apply(lambda x: pre_process(x))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,processed
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,"[nah, dont, think, goes, usf, lives, around, t..."


In [24]:
#categorizing ham/spam associated words

def categorize_words():
  spam_words = []
  ham_words = []

  #spam associated words
  for v2 in data['processed'][data['v1'] == 'spam']:
    for word in v2:
      spam_words.append(word)

  #ham associated words
  for v2 in data['processed'][data['v1'] == 'ham']:
    for word in v2:
      ham_words.append(word)

  return spam_words, ham_words

spam_words, ham_words = categorize_words()

print(spam_words[:5])
print(ham_words[:5])


['free', 'entry', '2', 'wkly', 'comp']
['go', 'jurong', 'point', 'crazy', 'available']


In [32]:
#iterate over all the words from the user input and count their occurances in both ham_words and spam_words

def predict(user_input):
  spam_counter = 0
  ham_counter = 0

  for word in user_input:
    spam_counter += spam_words.count(word)
    ham_counter += ham_words.count(word)

  print('*************************************RESULTS*************************************')
  if ham_counter > spam_counter:
    #adding accuracy
    accuracy = round((ham_counter / (ham_counter + spam_counter)) * 100, 2)
    print('message is not spam, with {}% accuracy'.format(accuracy))
  elif spam_counter > ham_counter:
    accuracy = round((spam_counter / (ham_counter + spam_counter)) * 100, 2)
    print('message is spam, with {}% accuracy'.format(accuracy) )
  else:
    print('message could be spam, with 50% accuracy')


In [42]:
#collect user input

user_input = input('please type a spam or ham message to check if our function predicts properly\n')

please type a spam or ham message to check if our function predicts properly
free call tday wkly srvc win $$$money


In [43]:
processed_input = pre_process(user_input)
predict(processed_input)

*************************************RESULTS*************************************
message is spam, with 64.55% accuracy
